In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [3]:
# load the dataset
dataset = pd.read_csv("life_expectancy.csv")
dataset = dataset.drop(["Country"], axis = 1)

# create labels and features
labels = dataset.iloc[:, -1]
features = dataset.iloc[:, 0:-1]

#OHE
features = pd.get_dummies(features)

#split train and test data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3, random_state = 42)

#Normalize the data
numerical_features = features.select_dtypes(include = ["float64", "int"])
numerical_columns = numerical_features.columns

ct = ColumnTransformer([("Std", StandardScaler(), numerical_columns)], remainder = "passthrough")

#fit the transformer 
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

#fit the model
my_model = Sequential()
input = InputLayer(shape = (features.shape[1], ))
my_model.add(input)
my_model.add(Dense(100, activation = "relu"))
my_model.add(Dense(1))

my_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │         2,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,301 (8.99 KB)

 Trainable params: 2,301 (8.99 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# add optimizer instance
opt = Adam(learning_rate = 0.1)

#compile the model 
my_model.compile(loss = "mse", metrics = ["mae"], optimizer = opt)

#fit the model 
my_model.fit(features_train_scaled, labels_train, epochs = 40, batch_size = 1, verbose = 1)

Epoch 1/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 362us/step - loss: 1478.0917 - mae: 20.8437
Epoch 2/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 365us/step - loss: 747.5928 - mae: 9.8596 
Epoch 3/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 369us/step - loss: 116.9718 - mae: 5.2747
Epoch 4/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 424us/step - loss: 42.6877 - mae: 4.5871
Epoch 5/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 347us/step - loss: 49.1623 - mae: 4.9706
Epoch 6/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 345us/step - loss: 32.9160 - mae: 4.1003
Epoch 7/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 359us/step - loss: 20.5571 - mae: 3.5019
Epoch 8/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 370us/step - loss: 28.1622 - mae: 3.8449
Epoch 9/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 377us/step - loss: 42.0427 - mae: 3.9239
Epoch 10/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 396us/step - loss: 22.6423 - mae: 3.6873
Epoch 11/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 345us/step - loss: 23.7580 - mae: 3.8156
Epoch 12/40
2056/2056 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [5]:
#evaluate the model 
res_mse, res_mae = my_model.evaluate(features_test_scaled, labels_test, verbose = 0)
print(f"The model mean squared error is: {res_mse}")
print(f"The model mean absolute error is {res_mae}")

The model mean squared error is: 26.332378387451172
The model mean absolute error is 3.322197914123535
